In [5]:
import pandas as pd
import ta 

dataset = pd.read_csv("aapl_5m_train.csv")
dataset = dataset.drop(columns=['Volume'])
dataset.head()

,Unnamed: 0,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close
0,0,1609770600,0,2021-01-04 14:30:00,133.570007,133.611602,132.389999,132.809997
1,1,1609770900,0,2021-01-04 14:35:00,132.750000,132.750000,131.809997,131.889999
2,2,1609771200,0,2021-01-04 14:40:00,131.500000,132.339996,131.500000,132.059997
3,3,1609771500,0,2021-01-04 14:45:00,132.000000,132.250000,131.899993,132.250000
4,4,1609771800,0,2021-01-04 14:50:00,132.000000,132.018096,131.520004,131.589996


In [7]:
lags = 7 

for lag in range (2, lags +1):
    dataset[f'lag_{lag}'] = dataset.Close.shift(lag)

dataset.head()

,Unnamed: 0,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7
0,0,1609770600,0,2021-01-04 14:30:00,133.570007,133.611602,132.389999,132.809997,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1609770900,0,2021-01-04 14:35:00,132.750000,132.750000,131.809997,131.889999,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1609771200,0,2021-01-04 14:40:00,131.500000,132.339996,131.500000,132.059997,132.809997,NaN,NaN,NaN,NaN,NaN
3,3,1609771500,0,2021-01-04 14:45:00,132.000000,132.250000,131.899993,132.250000,131.889999,132.809997,NaN,NaN,NaN,NaN
4,4,1609771800,0,2021-01-04 14:50:00,132.000000,132.018096,131.520004,131.589996,132.059997,131.889999,132.809997,NaN,NaN,NaN


In [11]:
rsi = ta.momentum.RSIIndicator(dataset.Close, window=23)
ultimate = ta.momentum.UltimateOscillator(
    high=dataset['High'], low=dataset['Low'], close=dataset['Close'],
    window1=7, window2=14, window3=28
)
williams = ta.momentum.WilliamsRIndicator(
    high=dataset['High'], low=dataset['Low'], close=dataset['Close'], lbp=14
)

dataset['RSI'] = rsi.rsi()
dataset['Ultimate'] = ultimate.ultimate_oscillator()
dataset['Williams'] = williams.williams_r()

dataset = dataset.dropna()
dataset.head()

,Unnamed: 0,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,RSI,Ultimate,Williams
28,28,1609779000,0,2021-01-04 16:50:00,129.410003,129.529998,129.110000,129.148498,129.390106,129.690002,129.910003,129.895004,129.996398,130.324996,26.272310,31.520554,-98.250089
29,29,1609779300,0,2021-01-04 16:55:00,129.130004,129.179992,128.789993,128.820007,129.413299,129.390106,129.690002,129.910003,129.895004,129.996398,24.310338,30.799220,-98.808970
30,30,1609779600,0,2021-01-04 17:00:00,128.809997,128.919296,128.399993,128.580001,129.148498,129.413299,129.390106,129.690002,129.910003,129.895004,22.998449,32.409321,-93.814167
31,31,1609779900,0,2021-01-04 17:05:00,128.596694,128.770004,128.330001,128.351699,128.820007,129.148498,129.413299,129.390106,129.690002,129.910003,21.827079,25.800252,-99.271878
32,32,1609780200,0,2021-01-04 17:10:00,128.233398,128.249893,127.400100,127.410003,128.580001,128.820007,129.148498,129.413299,129.390106,129.690002,17.896399,21.898425,-99.685599
